In [1]:
import os
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

# 数据准备

In [2]:
from dataset.dataset import load_cifar10

train_data,test_data=load_cifar10(batch_size=64)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(50000, 3072) (50000,)


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(10000, 3072) (10000,)


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# 模型搭建
为了便于更改关于调优的参数，如激活函数、初始化方法等，这里将之前的```mini_CNN```无脑打包成一个函数，暴露的参数只有```activation```与```initializer```。

In [3]:
import tensorflow as tf
from CNN.mini_CNN import mini_CNN

unit_I = train_data.n_features
X = tf.placeholder(tf.float32, [None, unit_I])
Y = tf.placeholder(tf.int64, [None])
X_img = tf.transpose(tf.reshape(
    X, [-1, 3, 32, 32]), perm=[0, 2, 3, 1])

with tf.name_scope('CNN'):
    logits = mini_CNN(X_img, activation=tf.nn.leaky_relu,
                      initializer=tf.initializers.he_uniform())

with tf.name_scope('Eval'):
    loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
    predict = tf.argmax(logits, 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), tf.float32))

with tf.name_scope('train_op'):
    lr = 1e-3
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)

init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


# 训练网络

In [4]:
with tf.Session(config=config) as sess:
    sess.run(init)
    epochs = 10

    batch_cnt = 0
    for epoch in range(epochs):
        for batch_data, batch_labels in train_data.next_batch():
            batch_cnt += 1
            loss_val, acc_val, _ = sess.run(
                [loss, accuracy, train_op], feed_dict={
                    X: batch_data,
                    Y: batch_labels})

    test_acc = sess.run(accuracy, feed_dict={
                        X: test_data.data,
                        Y: test_data.target})
    print('test_acc: {}'.format(test_acc))

test_acc: 0.7107999920845032


|activation|initializer|optimizer|acc|
|:-:|:-:|:-:|:-:|
|relu|None|AdamOptimizer|0.7256|
|relu|truncate_std0.02|AdamOptimizer|0.7083|
|relu|he_uniform|AdamOptimizer|0.7172|
|leaky_relu|None|AdamOptimizer|0.7275|
|leaky_relu|truncate_std0.02|AdamOptimizer|0.7280|
|leaky_relu|he_uniform|AdamOptimizer|0.7108|